In [12]:
!pip install geemap jupyter-contrib-nbextensions earthengine-api pycrs geopandas geedim

     ---------------------------------------- 0.0/23.5 MB ? eta -:--:--
     - -------------------------------------- 0.9/23.5 MB 18.7 MB/s eta 0:00:02
     ----- ---------------------------------- 3.4/23.5 MB 35.8 MB/s eta 0:00:01
     ---------- ----------------------------- 6.1/23.5 MB 42.8 MB/s eta 0:00:01
     --------------- ------------------------ 8.9/23.5 MB 47.1 MB/s eta 0:00:01
     ------------------ -------------------- 11.0/23.5 MB 54.4 MB/s eta 0:00:01
     --------------------- ----------------- 13.2/23.5 MB 54.4 MB/s eta 0:00:01
     -------------------------- ------------ 16.0/23.5 MB 59.5 MB/s eta 0:00:01
     ------------------------------- ------- 19.0/23.5 MB 59.8 MB/s eta 0:00:01
     ------------------------------------ -- 22.0/23.5 MB 59.5 MB/s eta 0:00:01
     --------------------------------------  23.5/23.5 MB 65.6 MB/s eta 0:00:01
     --------------------------------------  23.5/23.5 MB 65.6 MB/s eta 0:00:01
     --------------------------------------- 23.


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
import os
import ee
import geemap
from pathlib import Path

In [52]:
ee.Authenticate()
ee.Initialize(project="ee-yuweicao94")

In [53]:
#Source: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED

#*
 # Function to mask clouds using the Sentinel-2 QA band
 # @param {ee.Image} image Sentinel-2 image
 # @return {ee.Image} cloud masked Sentinel-2 image
 #
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

## RMF

In [54]:
import geopandas
import pyproj

#Set composite out dir
out_dir = r"D:\rfm_sen"

#Set output raster CRS as ESPG code
out_crs = "EPSG:2958"
#Create an interactive GEE map instance to view in notebook
Map = geemap.Map(basemap='CartoDB.Positron')

#Load study area
#study_area_fpath = Path(r"D:\Sync\research\tree species estimation\tree dataset\data_processing\FORMGMT\ROI_RMF\ROI_RMF_4326.shp")
#gdf = geopandas.read_file(study_area_fpath).to_crs("EPSG:2958")
#gdf.to_file(r"D:\Sync\research\tree species estimation\tree dataset\data_processing\FORMGMT\ROI_RMF\ROI_RMF_2958.shp")
study_area_fpath = Path(r"D:\Sync\research\tree species estimation\tree dataset\data_processing\FORMGMT\ROI_RMF\ROI_RMF_2958.shp")
study_area = geemap.shp_to_ee(study_area_fpath)

#Display the view to the center of the screen and scale the view
Map.centerObject(study_area, 9)

#Add study area to map
styling = {'color': "black", 'fillColor': '00000000'}
Map.addLayer(study_area.style(**styling), None, 'Study Area')

#View map
Map

Map(center=[48.24149220487604, -81.69277795078565], controls=(WidgetControl(options=['position', 'transparent_…

In [26]:
target_years=[2018,2019,2020,2021,2022]
for target_year in target_years:
    # Define the date ranges
    date_ranges = [
        (ee.Date.fromYMD(target_year-1, 12, 1), ee.Date.fromYMD(target_year, 3, 1)), # Winter
        (ee.Date.fromYMD(target_year, 3, 1), ee.Date.fromYMD(target_year, 5, 31)), # Spring
        (ee.Date.fromYMD(target_year, 6, 1), ee.Date.fromYMD(target_year, 8, 31)),  # Summer
        (ee.Date.fromYMD(target_year, 9, 1), ee.Date.fromYMD(target_year, 11, 30)), # Fall
    ]
    
    for startDate, endDate in date_ranges:
        # Get the month information
        start_month = startDate.format('MM').getInfo()
        end_month = endDate.format('MM').getInfo()

        # Map function across collection

        s2_composite = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                            .filterBounds(study_area) \
                            .filterDate(startDate, endDate) \
                            .map(maskS2clouds) \
                            .median() \
                            .clip(study_area)

        season = f'{start_month}-{end_month}'
        Map.addLayer(s2_composite, {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':0.3}, f'Sentinel-2 {target_year} Composite {season}')

Map.centerObject(study_area, 9)
Map

Map(bottom=45731.0, center=[48.24149220487412, -81.69277795078368], controls=(WidgetControl(options=['position…

In [ ]:
target_year=2018
#Set date range
startDate = ee.Date.fromYMD(target_year, 6, 1)
endDate = ee.Date.fromYMD(target_year, 8, 31)

#Map function across collection
s2_composite = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                    .filterBounds(study_area) \
                    .filterDate(startDate, endDate) \
                    .filter(ee.Filter.calendarRange(6, 8, 'month')) \
                    .map(maskS2clouds) \
                    .median() \
                    .clip(study_area)

# Check if the directory exists
if not os.path.exists(out_dir):
    # If it doesn't exist, create it
    os.makedirs(out_dir)

#Get study area geometry
roi = study_area.geometry()

#Clip image to specific bounding geometry of study area
s2_composite = s2_composite.clip(roi).unmask()

#Export S2
s2_out_fpath = os.path.join(out_dir, f"s2_comp_{target_year}.tif")
geemap.download_ee_image(s2_composite, filename=s2_out_fpath, scale=10, region=roi, crs=out_crs)

Consider adjusting `region`, `scale` and/or `dtype` to reduce the s2_comp_2018.tif download size (raw: 12.55 GB).


s2_comp_2018.tif: |                                                   | 0.00/12.6G (raw) [  0.0%] in 00:00 (et…

In [16]:
#Set date range
target_year=2018
startDate = ee.Date.fromYMD(target_year, 9, 1)
endDate = ee.Date.fromYMD(target_year, 11, 30)

#Map function across collection
s2_composite = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                    .filterBounds(study_area) \
                    .filterDate(startDate, endDate) \
                    .map(maskS2clouds) \
                    .median() \
                    .clip(study_area)
                    
# Check if the directory exists
if not os.path.exists(out_dir):
    # If it doesn't exist, create it
    os.makedirs(out_dir)

#Get study area geometry
roi = study_area.geometry()

#Clip image to specific bounding geometry of study area
s2_composite = s2_composite.clip(roi).unmask()

#Export S2
s2_out_fpath = os.path.join(out_dir, f"s2_comp_{target_year}_9-11.tif")
geemap.download_ee_image(s2_composite, filename=s2_out_fpath, scale=10, region=roi, crs=out_crs)

AttributeError: 'ImageCollection' object has no attribute 'clip'

In [17]:
s2_composite

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


## ovf

In [15]:
from pathlib import Path
import geopandas

#Set composite out dir
out_dir = r"D:\ovf_sen"

#Set output raster CRS as ESPG code
out_crs = "EPSG:2958"

#Load study area
study_area_fpath = Path(r"D:\Sync\research\tree species estimation\tree dataset\data_processing\FORMGMT\ROI_OVF\ROI_OVF.shp")
gdf = geopandas.read_file(study_area_fpath).to_crs("EPSG:2958")
gdf.to_file(r"D:\Sync\research\tree species estimation\tree dataset\data_processing\FORMGMT\ROI_OVF\ROI_OVF_2958.shp")
study_area_fpath = Path(r"D:\Sync\research\tree species estimation\tree dataset\data_processing\FORMGMT\ROI_OVF\ROI_OVF_2958.shp")
study_area = geemap.shp_to_ee(study_area_fpath)

#Display the view to the center of the screen and scale the view
Map.centerObject(study_area, 9)

#Add study area to map
styling = {'color': "black", 'fillColor': '00000000', 'width': 6}
Map.addLayer(study_area.style(**styling), None, 'Study Area')

#View map
Map

Map(bottom=45731.0, center=[45.614631634514424, -77.26736638534793], controls=(WidgetControl(options=['positio…

In [7]:
target_years = [2019, 2020, 2021]

In [8]:
for target_year in target_years:
    # Define the date ranges
    date_ranges = [
        (ee.Date.fromYMD(target_year-1, 12, 1), ee.Date.fromYMD(target_year, 3, 1)), # Winter
        (ee.Date.fromYMD(target_year, 3, 1), ee.Date.fromYMD(target_year, 5, 31)), # Spring
        (ee.Date.fromYMD(target_year, 6, 1), ee.Date.fromYMD(target_year, 8, 31)),  # Summer
        (ee.Date.fromYMD(target_year, 9, 1), ee.Date.fromYMD(target_year, 11, 30)), # Fall
    ]
    
    for startDate, endDate in date_ranges:
        # Get the month information
        start_month = startDate.format('MM').getInfo()
        end_month = endDate.format('MM').getInfo()

        # Map function across collection
        s2_composite = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                            .filterBounds(study_area) \
                            .filterDate(startDate, endDate) \
                            .map(maskS2clouds) \
                            .median() \
                            .clip(study_area)

        season = f'{start_month}-{end_month}'
        Map.addLayer(s2_composite, {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':0.3}, f'Sentinel-2 {target_year} Composite {season}')

        # Check if the directory exists
        if not os.path.exists(out_dir):
            # If it doesn't exist, create it
            os.makedirs(out_dir)

        # Get study area geometry
        roi = study_area.geometry()

        # Clip image to specific bounding geometry of study area
        s2_composite = s2_composite.clip(roi).unmask()

        # Export S2
        s2_out_fpath = os.path.join(out_dir, f"s2_comp_{target_year}_{season}.tif")
        geemap.download_ee_image(s2_composite, filename=s2_out_fpath, scale=10, region=roi, crs=out_crs)
Map

Consider adjusting `region`, `scale` and/or `dtype` to reduce the s2_comp_2019_12-03.tif download size (raw: 22.57 GB).


s2_comp_2019_12-03.tif: |                                             | 0.00/22.6G (raw) [  0.0%] in 00:00 (et…

There is no STAC entry for: None
Consider adjusting `region`, `scale` and/or `dtype` to reduce the s2_comp_2019_03-05.tif download size (raw: 22.57 GB).


s2_comp_2019_03-05.tif: |                                             | 0.00/22.6G (raw) [  0.0%] in 00:00 (et…

Consider adjusting `region`, `scale` and/or `dtype` to reduce the s2_comp_2019_06-08.tif download size (raw: 22.57 GB).


...

Consider adjusting `region`, `scale` and/or `dtype` to reduce the s2_comp_2021_06-08.tif download size (raw: 22.57 GB).


s2_comp_2021_06-08.tif: |                                             | 0.00/22.6G (raw) [  0.0%] in 00:00 (et…

Consider adjusting `region`, `scale` and/or `dtype` to reduce the s2_comp_2021_09-11.tif download size (raw: 22.57 GB).


s2_comp_2021_09-11.tif: |                                             | 0.00/22.6G (raw) [  0.0%] in 00:00 (et…

Map(bottom=23648.0, center=[45.87471224890479, -77.82989501953126], controls=(WidgetControl(options=['position…

In [23]:
!pip install openeo

Defaulting to user installation because normal site-packages is not writeable
  Using cached pywin32-306-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
   ---------------------------------------- 265.6/265.6 kB 5.4 MB/s eta 0:00:00
   --------------------------------------- 182.9/182.9 kB 11.5 MB/s eta 0:00:00
Using cached pywin32-306-cp311-cp311-win_amd64.whl (9.2 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [24]:
import openeo

connection = openeo.connect("openeo.dataspace.copernicus.eu")

In [25]:
# List collections available on the openEO back-end
connection.list_collection_ids()

# Get detailed metadata of a certain collection
connection.describe_collection("SENTINEL2_L2A")

{'assets': {'thumbnail': {'href': 'https://www.esa.int/var/esa/storage/images/esa_multimedia/images/2022/07/tallinn_estonia/24383281-1-eng-GB/Tallinn_Estonia_pillars.jpg',
   'roles': ['thumbnail'],
   'title': 'Sentinel 2 L2A',
   'type': 'image/png'}},
 'cube:dimensions': {'bands': {'type': 'bands',
   'values': ['B01',
    'B02',
    'B03',
    'B04',
    'B05',
    'B06',
    'B07',
    'B08',
    'B8A',
    'B09',
    'B11',
    'B12',
    'WVP',
    'AOT',
    'SCL',
    'sunAzimuthAngles',
    'sunZenithAngles',
    'viewAzimuthMean',
    'viewZenithMean']},
  't': {'extent': ['2015-07-04T00:00:00Z', None], 'type': 'temporal'},
  'x': {'axis': 'x',
   'extent': [-180, 180],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'Lat',
       'direction': 'north',
       'name': 'Geodetic latitude',
       'unit': 'degree'},
      {'abbreviation': 'Lon',
       'direction': 'east',
       'name': 'Geodetic longitude',
       'unit': 'degree'}],
     'subtype': 'ellipsoidal'},
    'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
      'name': 'WGS 84',
      'semi_major_axis': 6378137},
     'name': 'World Geodetic System 1984',
     'type': 'GeodeticReferenceFrame'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'GeodeticCRS'},
   'step': 10,
   'type': 'spatial'},
  'y': {'axis': 'y',
   'extent': [-56, 83],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'Lat',
       'direction': 'north',
       'name': 'Geodetic latitude',
       'unit': 'degree'},
      {'abbreviation': 'Lon',
       'direction': 'east',
       'name': 'Geodetic longitude',
       'unit': 'degree'}],
     'subtype': 'ellipsoidal'},
    'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
      'name': 'WGS 84',
      'semi_major_axis': 6378137},
     'name': 'World Geodetic System 1984',
     'type': 'GeodeticReferenceFrame'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'GeodeticCRS'},
   'step': 10,
   'type': 'spatial'}},
 'description': 'SENTINEL-2 is a wide-swath, high-resolution, multi-spectral imaging mission, supporting Copernicus Land Monitoring studies, including the monitoring of vegetation, soil and water cover, as well as observation of inland waterways and coastal areas. The SENTINEL-2 Multispectral Instrument (MSI) samples 13 spectral bands: four bands at 10 metres, six bands at 20 metres and three bands at 60 metres spatial resolution.\n The level 2A data is atmospherically corrected using Sen2Cor.',
 'extent': {'spatial': {'bbox': [[-180, -56, 180, 83]]},
  'temporal': {'interval': [['2015-07-04T00:00:00Z', None]]}},
 'id': 'SENTINEL2_L2A',
 'keywords': ['COPERNICUS',
  'ESA',
  'Orthoimagery',
  'Sentinel-2',
  'MSI',
  'Level-2A',
  'Radiometry',
  'Plant Resource',
  'TOC'],
 'license': 'proprietary',
 'links': [{'href': 'https://sentinels.copernicus.eu/documents/247904/690755/Sentinel_Data_Legal_Notice',
   'rel': 'license',
   'type': 'application/pdf'},
  {'href': 'https://documentation.dataspace.copernicus.eu/Data/SentinelMissions/Sentinel2.html#sentinel-2-level-2a-top-of-canopy-toc',
   'rel': 'alternate',
   'type': 'application/html'},
  {'href': 'https://dataspace.copernicus.eu/browser/?zoom=11&lat=45.36638&lng=12.49832&themeId=DEFAULT-THEME&visualizationUrl=https%3A%2F%2Fsh.dataspace.copernicus.eu%2Fogc%2Fwms%2F28b654e7-8912-4e59-9e58-85b58d768b3a&datasetId=S2

In [26]:
connection.authenticate_oidc()

Visit https://identity.dataspace.copernicus.eu/auth/realms/CDSE/device?user_code=TFMP-UYIO 📋 to authenticate.

✅ Authorized successfully

Authenticated using device code flow.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

In [50]:
import geopandas as gpd
from pathlib import Path

gdf = gpd.read_file(Path(r"D:\Sync\research\tree species estimation\tree dataset\data_processing\FORMGMT\ROI_RMF\ROI_RMF_4326_wgs.geojson"))
area = gdf.geometry[0]

temporal_extent = ["2018-09-01", "2018-11-30"]
max_cloud_cover = 70
spatial_resolution = 20

In [46]:
import openeo
import numpy as np

from openeo.processes import if_, is_nan

from utils_BAP import (calculate_cloud_mask, calculate_cloud_coverage_score,
                           calculate_date_score, calculate_distance_to_cloud_score,
                           calculate_distance_to_cloud_score, aggregate_BAP_scores,
                           create_rank_mask)

ModuleNotFoundError: No module named 'utils_BAP'

In [ ]:
sentinel2_cube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent={"west": -82.3805033, "south": 47.7544353999999984, "east": -80.985631499999996748, "north": 48.7986475000000013},
    temporal_extent = ["2018-09-01", "2018-11-30"],
    max_cloud_cover=15,
).filter_spatial(area)

In [39]:
#From this data cube, we can now select the individual bands with the DataCube.band() method and rescale the digital number values to physical reflectances:
blue = sentinel2_cube.band("B02") * 0.0001
red = sentinel2_cube.band("B04") * 0.0001
nir = sentinel2_cube.band("B08") * 0.0001

# We now want to compute the enhanced vegetation index and can do that directly with these band variables:
evi_cube = 2.5 * (nir - red) / (nir + 6.0 * red - 7.5 * blue + 1.0)

# Now we can use the compact “band math” feature again to build a binary mask with a simple comparison operation:
# Select the "SCL" band from the data cube
scl_band = sentinel2_cube.band("SCL")
# Build mask to mask out everything but class 4 (vegetation)
mask = (scl_band != 4)

# Before we can apply this mask to the EVI cube we have to resample it, as the “SCL” layer has a “ground sample distance” of 20 meter, while it is 10 meter for the “B02”, “B04” and “B08” bands. We can easily do the resampling by referring directly to the EVI cube.
mask_resampled = mask.resample_cube_spatial(evi_cube)

# Apply the mask to the `evi_cube`
evi_cube_masked = evi_cube.mask(mask_resampled)

# Because GeoTIFF does not support a temporal dimension, we first eliminate it by taking the temporal maximum value for each pixel:
evi_composite = evi_cube.max_time()

# Now we can download this to a local file:
evi_composite.download("evi-composite.tiff")

OpenEoApiError: [500] Internal: Server error: Exception during Spark execution: org.apache.spark.SparkException: Job 3807 cancelled runaway job 3807 cancelled after PT15M (ref: r-240827de7ee6473c82053049c7b61aa6)

In [21]:
# Utilities
import matplotlib.pyplot as plt
import pandas as pd
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    SentinelHubStatistical,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

#from utils import plot_image

ModuleNotFoundError: No module named 'utils'

In [22]:
# Only run this cell if you have not created a configuration.

config = SHConfig()
# config.sh_client_id = getpass.getpass("Enter your SentinelHub client id")
# config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
# config.save("cdse")

In [ ]:
aoi_coords_wgs84 = [12.292349, 47.810849, 12.569037, 47.967123]